In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../"))

# Any results you write to the current directory are saved as output.

['Toxic', 'kaggle-competitions']


## check files

In [2]:
from subprocess import check_output
print(check_output(["ls", "./"]).decode('utf8'))

glove.6B.50d.txt
glove.840B.300d.txt
glove.840B.300d.txt.zip
notebook.ipynb
Replacers.py
sample_submission.csv
submission.csv
submission_version-1(LSTM).csv
submission_version-2(LSTM+tanh+softmax).csv
submission_version-3_GRU.csv
submission_version-4_CNN.csv
submission_version-5_GRU.csv
submission_version-6_LSTM+ConvNet.csv
submission_version-7+GRU_Pre.csv
submission_version-8+GRU_withoutPre.csv
test.csv
Test_witout_stop.csv
Toxic_Prediction_with_keras.ipynb
Toxic_Prediction_with_keras_VERSION-01.ipynb
Toxic_Solution.ipynb
train.csv
Train_without_stop.csv



## Data reading

In [3]:
import os,re,csv,codecs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
list_sentences_train = train['comment_text'].fillna(' ').values
list_sentences_test = test['comment_text'].fillna(' ').values 

In [6]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [7]:
y = train[list_classes].values

## Tokenize the data 

In [8]:
#max_features = 30000
max_features = 110000
maxlen =  150
#embed_size = 50
embed_size = 300

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

Using TensorFlow backend.


In [10]:
tokenizer = Tokenizer(num_words = max_features)

In [11]:
tokenizer.fit_on_texts(list(list_sentences_train))

In [12]:
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [13]:
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [14]:
print(X_train.shape, X_test.shape)

(159571, 150) (153164, 150)


## Making text to vector using GloVE 

### Read the GloVe

In [15]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float32)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i += 1
        return words_to_index, index_to_words, word_to_vec_map

In [16]:
glove_file_50d = f'glove.6B.50d.txt'
#glove_file_300d = f'../input/glove840b300dtxt/glove.840B.300d.txt'
words_to_index, index_to_words, word_to_vec_map = read_glove_vecs(glove_file_50d)

In [17]:
print(len(words_to_index), len(index_to_words), len(word_to_vec_map))

400000 400000 400000


In [18]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

In [19]:
glove_file_300d = f'glove.840B.300d.txt'
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(glove_file_300d))

In [20]:
#embedding_index

In [21]:
len(embedding_index)

2196008

### Vectorize - 50d

In [22]:
import numpy as np

In [23]:
#all_embs = np.stack(word_to_vec_map.values())

In [24]:
#print(all_embs)

In [25]:
#emb_mean, emb_std = all_embs.mean(), all_embs.std()

In [26]:
#word_index = tokenizer.word_index

In [27]:
#word_index

In [28]:
#nb_words = min(max_features, len(word_index))

In [29]:
#nb_words

In [30]:
# Making embedding_matrix
#embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
#embedding_matrix.shape

In [31]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))

In [32]:
for w, i in words_to_index.items():
    if i >= max_features:
        continue
    #embedding_vector = word_to_vec_map.get(w) # 50d 
    embedding_vector = embedding_index.get(w) # 300d
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [33]:
len(embedding_matrix)

110000

## Import necessary libraries

In [34]:
# keras libraries
from keras.models import Model, load_model,Sequential
from keras.layers import Dense, Input, Dropout,Bidirectional, GRU, Activation, concatenate, Embedding, SpatialDropout1D
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D ,GlobalMaxPool1D, GlobalAvgPool1D
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence, text
from keras import layers


## GRU

In [35]:
def get_model_gru():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x) # 0.2 -> 0.1
    x = Bidirectional(GRU(80, return_sequences=True, activation='relu', dropout=0.1, recurrent_dropout=0.0))(x) # 80 -> 85
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

    return model


In [36]:
model = get_model_gru()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 300)     33000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 150, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 150, 160)     182880      spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
global_ave

In [37]:
model.fit(X_train, y, epochs=5, batch_size=32, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
 24128/143613 [====>.........................] - ETA: 1:04:57 - loss: 0.1010 - acc: 0.9692

KeyboardInterrupt: 

In [ ]:
y_test = model.predict([X_test], batch_size=1024, verbose=1)

In [ ]:
sample_submission = f'sample_submission.csv'
sample_submission = pd.read_csv(sample_submission)

In [ ]:
sample_submission[list_classes] = y_test
sample_submission.to_csv('submission_version-9+GRU.csv', index=False)

In [ ]:
model.summary()